In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 10.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "gkf_fold"

boosting_model_names = ["xgb", "xgb", "xgb", "lgb", "cb"]
boosting_versions = ["v1", "v3", "v5", "v6", "v1"]
boosting_modes = ["train", "train", "train", "train", "train"]

cnn_model_names = ["efficientnet_b0", "efficientnet_b1", "tf_efficientnet_b1_ns", "mobilevitv2_200"]
cnn_versions = ["v2", "v1", "v1", "v1"]
cnn_modes = ["trainbinary", "trainbinary", "trainbinary", "trainbinary"]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
oof_columns = []
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_name}_{version}.csv")
    if mode in ["trainbinary", "trainmulti"]:
        oof_column = f"oof_{model_name}_{version}_{mode}"
        oof_preds_model_df = oof_preds_model_df.rename(columns={f"oof_{model_name}_{version}": oof_column})
    else:
        oof_column = f"oof_{model_name}_{version}"
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, oof_column]], on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}_{mode}"] = {}
    val_pauc_scores[f"{model_name}_{version}_{mode}"] = {}
    oof_columns.append(oof_column)

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |    w5     |    w6     |    w7     |    w8     |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.184     | 0.09359   | 4.991     | 1.134     | 0.4997    | 6.854     | 4.87      | 8.977     | 6.475     | 8.97      |
| 2         | 0.1851    | 7.211     | 8.314     | 8.276     | 8.336     | 9.57      | 3.68      | 4.948     | 3.395     | 6.194     |
| 3         | 0.1846    | 9.775     | 0.9643    | 7.442     | 2.925     | 2.987     | 7.525     | 0.1866    | 5.237     | 8.644     |
| 4         | 0.1847    | 3.888     | 2.122     | 4.752     | 5.647     | 3.494     | 9.759     | 0.3782    | 7.943     | 3.579     |
| 5         | 0.185     | 7.48      | 9.145     | 3.727     | 9.649     | 0.8139    | 0.4245    | 2.968     | 3.637     | 4.903     |
| 6         | 0.1846    | 6.685     | 6.734     | 5.721     | 

In [4]:
pprint(oof_columns)

['oof_xgb_v1',
 'oof_xgb_v3',
 'oof_xgb_v5',
 'oof_lgb_v6',
 'oof_cb_v1',
 'oof_efficientnet_b0_v2_trainbinary',
 'oof_efficientnet_b1_v1_trainbinary',
 'oof_tf_efficientnet_b1_ns_v1_trainbinary',
 'oof_mobilevitv2_200_v1_trainbinary']


In [5]:
pprint(weights)

[8.031281440024475,
 9.363810434324664,
 4.419550665723647,
 8.09402809347118,
 3.26907802191411,
 4.487310223293477,
 1.8218479149122448,
 6.421285339801205,
 1.7069076106897372]


In [6]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, oof_column]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
    print(f"Model: {model_name}_{version}_{mode} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}_{mode}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}_{mode}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[oof_column])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[oof_column], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")
print(f"CV PAUC RANGE: ({cv_ensemble_pauc_avg - cv_ensemble_pauc_std}, {cv_ensemble_pauc_avg + cv_ensemble_pauc_std})")

Model: xgb_v1_train | Weightage: 8.031281440024475
Val AUC scores:
{'fold_1': 0.9885459138238754,
 'fold_2': 0.9750098872849515,
 'fold_3': 0.9644496199972544,
 'fold_4': 0.9831541475833323,
 'fold_5': 0.9804010128033939}
Val PAUC scores:
{'fold_1': 0.1913092984108869,
 'fold_2': 0.17974234503360711,
 'fold_3': 0.16975983077710247,
 'fold_4': 0.18545257141430904,
 'fold_5': 0.184016348461708}
CV AUC OOF: 0.9325410500676679
CV PAUC OOF: 0.13681816531200738
CV AUC AVG: 0.9783121162985614
CV PAUC AVG: 0.1820560788195227
CV AUC STD: 0.008192836963125776
CV PAUC STD: 0.007177074219698841


Model: xgb_v3_train | Weightage: 9.363810434324664
Val AUC scores:
{'fold_1': 0.9789757767096575,
 'fold_2': 0.9653996222993153,
 'fold_3': 0.9664791092086708,
 'fold_4': 0.9677780759007127,
 'fold_5': 0.9726160386562468}
Val PAUC scores:
{'fold_1': 0.1851343535234871,
 'fold_2': 0.17305110094437967,
 'fold_3': 0.17426167775267995,
 'fold_4': 0.17145455566509835,
 'fold_5': 0.17612597262179827}
CV AUC OOF

In [7]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v2_trainbinary,oof_efficientnet_b1_v1_trainbinary,oof_ensemble,oof_lgb_v6,oof_mobilevitv2_200_v1_trainbinary,oof_tf_efficientnet_b1_ns_v1_trainbinary,oof_xgb_v1,oof_xgb_v3,oof_xgb_v5
target,,,,,,,,,,
0,0.009899,0.00229,0.000992,23.785764,0.011029,0.001209,0.001437,0.004694,0.010824,0.022984
1,0.468573,0.32989,0.230465,46.321080,0.482563,0.296823,0.272554,0.177280,0.337296,0.345672


In [8]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v1,oof_xgb_v3,oof_xgb_v5,oof_lgb_v6,oof_cb_v1,oof_efficientnet_b0_v2_trainbinary,oof_efficientnet_b1_v1_trainbinary,oof_tf_efficientnet_b1_ns_v1_trainbinary,oof_mobilevitv2_200_v1_trainbinary,oof_ensemble
oof_xgb_v1,1.000000,0.599760,0.509427,0.775729,0.823548,0.705345,0.645102,0.629036,0.615970,0.216778
oof_xgb_v3,0.599760,1.000000,0.754328,0.708221,0.718473,0.500615,0.419937,0.387086,0.410881,0.329461
oof_xgb_v5,0.509427,0.754328,1.000000,0.625106,0.636613,0.429455,0.357309,0.330237,0.364992,0.340358
oof_lgb_v6,0.775729,0.708221,0.625106,1.000000,0.955985,0.615857,0.496669,0.517019,0.501281,0.349778
oof_cb_v1,0.823548,0.718473,0.636613,0.955985,1.000000,0.669240,0.550353,0.570036,0.548923,0.342389
oof_efficientnet_b0_v2_trainbinary,0.705345,0.500615,0.429455,0.615857,0.669240,1.000000,0.765765,0.726273,0.713612,0.149771
oof_efficientnet_b1_v1_trainbinary,0.645102,0.419937,0.357309,0.496669,0.550353,0.765765,1.000000,0.742009,0.754796,0.091837
oof_tf_efficientnet_b1_ns_v1_trainbinary,0.629036,0.387086,0.330237,0.517019,0.570036,0.726273,0.742009,1.000000,0.734950,0.111060
oof_mobilevitv2_200_v1_trainbinary,0.615970,0.410881,0.364992,0.501281,0.548923,0.713612,0.754796,0.734950,1.000000,0.095771
oof_ensemble,0.216778,0.329461,0.340358,0.349778,0.342389,0.149771,0.091837,0.111060,0.095771,1.000000
